# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import re

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import original data and preprocessed data

In [3]:
#sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
#calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

In [4]:
sales_train_val = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\sales_train_validation.csv')
calendar = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [5]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [6]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
568,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,0,5,0,0,0,0,0,0,1,0
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10754,HOUSEHOLD_2_512_CA_4_validation,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
10755,HOUSEHOLD_2_513_CA_4_validation,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [7]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Household\Demand Pattern Categorization'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [8]:
len(sales_intermittent)

1493

In [9]:
len(sales_lumpy)

178

In [10]:
len(sales_erratic)

126

In [11]:
len(sales_smooth)

2391

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [12]:
sales_lumpy

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
579,HOUSEHOLD_1_015_CA_1_validation,HOUSEHOLD_1_015,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
581,HOUSEHOLD_1_017_CA_1_validation,HOUSEHOLD_1_017,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
596,HOUSEHOLD_1_033_CA_1_validation,HOUSEHOLD_1_033,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,2,3,3,...,0,0,2,2,1,2,1,2,4,5
704,HOUSEHOLD_1_144_CA_1_validation,HOUSEHOLD_1_144,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,2,1,1,1,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10556,HOUSEHOLD_2_314_CA_4_validation,HOUSEHOLD_2_314,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
10560,HOUSEHOLD_2_318_CA_4_validation,HOUSEHOLD_2_318,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10582,HOUSEHOLD_2_340_CA_4_validation,HOUSEHOLD_2_340,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10599,HOUSEHOLD_2_357_CA_4_validation,HOUSEHOLD_2_357,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,1,1,1,0,0,0,1,0


# User-defined functions to calculate Metrics

In [13]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)


def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

# Format sales data and then merge with calendar data

In [14]:
sales_pattern = sales_lumpy.copy()

In [15]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [16]:
dataframe_weekly

,wm_yr_wk,HOUSEHOLD_1_001_CA_4,HOUSEHOLD_1_002_CA_1,HOUSEHOLD_1_004_CA_4,HOUSEHOLD_1_013_CA_3,HOUSEHOLD_1_015_CA_1,HOUSEHOLD_1_015_CA_2,HOUSEHOLD_1_017_CA_1,HOUSEHOLD_1_020_CA_3,HOUSEHOLD_1_032_CA_3,...,HOUSEHOLD_2_436_CA_1,HOUSEHOLD_2_448_CA_1,HOUSEHOLD_2_479_CA_1,HOUSEHOLD_2_479_CA_2,HOUSEHOLD_2_479_CA_3,HOUSEHOLD_2_482_CA_3,HOUSEHOLD_2_484_CA_3,HOUSEHOLD_2_487_CA_1,HOUSEHOLD_2_504_CA_1,HOUSEHOLD_2_504_CA_2
0,11101,1.0,2.0,5.0,12.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,3.0,0.0,6.0,0.0,0.0,0.0
1,11102,0.0,1.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0
2,11103,0.0,5.0,0.0,49.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,3.0,7.0,0.0,3.0,0.0,0.0,1.0
3,11104,0.0,1.0,0.0,20.0,0.0,3.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,3.0,0.0,3.0,0.0,1.0,0.0
4,11105,0.0,0.0,2.0,26.0,0.0,1.0,0.0,0.0,0.0,...,2.0,0.0,1.0,2.0,3.0,0.0,7.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,1.0,3.0,1.0,14.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,3.0,3.0,9.0,0.0,0.0,0.0,0.0,0.0
269,11609,0.0,0.0,2.0,10.0,1.0,0.0,0.0,0.0,0.0,...,3.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0
270,11610,0.0,1.0,0.0,20.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,3.0,1.0,6.0,0.0,0.0,0.0,2.0,0.0
271,11611,1.0,2.0,2.0,17.0,1.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0


In [17]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

# Run ARIMA model with best combination

In [18]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [19]:
len(list_pattern_py)

178

In [20]:
best_combination = (2, 1, 0)

In [21]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    
    train_product = train_data[product]
    test_product = test_data[product]
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()
    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Product': [product for product_count in range(len(test_product))],
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOUSEHOLD_1_002_CA_1
Progressing: 0.0 %
Currently Running Product: HOUSEHOLD_1_015_CA_1
Progressing: 0.56 %
Currently Running Product: HOUSEHOLD_1_017_CA_1
Progressing: 1.12 %
Currently Running Product: HOUSEHOLD_1_033_CA_1
Progressing: 1.69 %
Currently Running Product: HOUSEHOLD_1_144_CA_1
Progressing: 2.25 %
Currently Running Product: HOUSEHOLD_1_157_CA_1
Progressing: 2.81 %
Currently Running Product: HOUSEHOLD_1_189_CA_1
Progressing: 3.37 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOUSEHOLD_1_199_CA_1
Progressing: 3.93 %
Currently Running Product: HOUSEHOLD_1_237_CA_1
Progressing: 4.49 %
Currently Running Product: HOUSEHOLD_1_263_CA_1
Progressing: 5.06 %
Currently Running Product: HOUSEHOLD_1_264_CA_1
Progressing: 5.62 %
Currently Running Product: HOUSEHOLD_1_297_CA_1
Progressing: 6.18 %
Currently Running Product: HOUSEHOLD_1_414_CA_1
Progressing: 6.74 %
Currently Running Product: HOUSEHOLD_1_470_CA_1
Progressing: 7.3 %
Currently Running Product: HOUSEHOLD_2_051_CA_1
Progressing: 7.87 %
Currently Running Product: HOUSEHOLD_2_053_CA_1
Progressing: 8.43 %
Currently Running Product: HOUSEHOLD_2_067_CA_1
Progressing: 8.99 %
Currently Running Product: HOUSEHOLD_2_080_CA_1
Progressing: 9.55 %
Currently Running Product: HOUSEHOLD_2_083_CA_1
Progressing: 10.11 %
Currently Running Product: HOUSEHOLD_2_100_CA_1
Progressing: 10.67 %
Currently Running Product: HOUSEHOLD_2_111_CA_1
Progressing: 11.24 %
Currently Running Product: HOUSEHOLD_2_128_CA_

C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOUSEHOLD_2_129_CA_1
Progressing: 12.36 %
Currently Running Product: HOUSEHOLD_2_132_CA_1
Progressing: 12.92 %
Currently Running Product: HOUSEHOLD_2_190_CA_1
Progressing: 13.48 %
Currently Running Product: HOUSEHOLD_2_192_CA_1
Progressing: 14.04 %
Currently Running Product: HOUSEHOLD_2_216_CA_1
Progressing: 14.61 %


...

C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOUSEHOLD_1_523_CA_4
Progressing: 89.89 %
Currently Running Product: HOUSEHOLD_2_013_CA_4
Progressing: 90.45 %
Currently Running Product: HOUSEHOLD_2_045_CA_4
Progressing: 91.01 %
Currently Running Product: HOUSEHOLD_2_057_CA_4
Progressing: 91.57 %
Currently Running Product: HOUSEHOLD_2_067_CA_4
Progressing: 92.13 %
Currently Running Product: HOUSEHOLD_2_080_CA_4
Progressing: 92.7 %
Currently Running Product: HOUSEHOLD_2_103_CA_4
Progressing: 93.26 %


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

Currently Running Product: HOUSEHOLD_2_111_CA_4
Progressing: 93.82 %
Currently Running Product: HOUSEHOLD_2_128_CA_4
Progressing: 94.38 %
Currently Running Product: HOUSEHOLD_2_190_CA_4
Progressing: 94.94 %
Currently Running Product: HOUSEHOLD_2_237_CA_4
Progressing: 95.51 %
Currently Running Product: HOUSEHOLD_2_240_CA_4
Progressing: 96.07 %
Currently Running Product: HOUSEHOLD_2_302_CA_4
Progressing: 96.63 %
Currently Running Product: HOUSEHOLD_2_314_CA_4
Progressing: 97.19 %
Currently Running Product: HOUSEHOLD_2_318_CA_4
Progressing: 97.75 %
Currently Running Product: HOUSEHOLD_2_340_CA_4
Progressing: 98.31 %
Currently Running Product: HOUSEHOLD_2_357_CA_4
Progressing: 98.88 %
Currently Running Product: HOUSEHOLD_2_418_CA_4
Progressing: 99.44 %
This line of code took 0.09748574999999994 minutes


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base

# Calculate metrics for optimal combination

In [22]:
# Start calculating metrics 
start = timer()

df_arima_metrics = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.004785181666666697 minutes


In [23]:
df_arima_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_4,0.369230,73.8460,148.9449,65.7222,3.692302e-01,4.682630e-01
1,HOUSEHOLD_1_002_CA_1,0.468682,93.7364,135.9961,52.9370,9.373639e-01,9.451894e-01
2,HOUSEHOLD_1_004_CA_4,0.477082,95.4165,134.9152,51.7587,9.541646e-01,9.575974e-01
3,HOUSEHOLD_1_013_CA_3,0.387662,23.8561,25.5599,33.2797,3.101297e+00,4.079449e+00
4,HOUSEHOLD_1_015_CA_1,0.421015,84.2030,117.4155,29.6648,4.210149e-01,4.389997e-01
...,...,...,...,...,...,...,...
173,HOUSEHOLD_2_482_CA_3,inf,inf,200.0000,NaN,9.779919e-17,9.823740e-17
174,HOUSEHOLD_2_484_CA_3,0.500000,100.0000,200.0000,100.0000,1.000000e+00,1.414214e+00
175,HOUSEHOLD_2_487_CA_1,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
176,HOUSEHOLD_2_504_CA_1,inf,inf,200.0000,NaN,1.016407e+00,1.045454e+00


In [24]:
df_arima_metrics.MAPE.mean()

86.42717549019606

In [25]:
df_arima_metrics.RMSE.mean()

1.790833716080676

In [27]:
df_arima_metrics.to_csv('ARIMA_Lumpy_Forecast_Metrics.csv')